In [42]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose, Concatenate, Input
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [43]:
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [44]:
batch_size = 8
lr = 1e-4
epochs = 100
height = 1450
width = 1340

In [45]:
dataset_path = os.path.join("dataset","non-aug")
files_dir = os.path.join("files","non-aug")

model_file = os.path.join(files_dir,"unet-non-aug.h5")
log_file = os.path.join(files_dir, "log-non-aug.csv")

In [46]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [47]:
create_dir(files_dir)

### *Building Unet*

Conv Block

In [48]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters,3,padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters,3,padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

Encoder Block

In [49]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPooling2D(2, 2)(x)
    return x,p

Decoder Block

In [50]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters,(2,2),strides=2,padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)

    return x

UNET

In [51]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2,256)
    s4, p4 = encoder_block(p3,512)

    """ Bridge """
    b1 = conv_block(p4, 1024)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    model = Model(inputs,outputs, name = "UNET")

    return model

Dataset Pipeline

In [52]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*")))
    train_y = sorted(glob(os.path.join(path, "train", "mask", "*")))

    valid_x = sorted(glob(os.path.join(path, "valid", "images", "*")))
    valid_y = sorted(glob(os.path.join(path, "valid", "mask", "*")))

    return (train_x,train_y), (valid_x,valid_y)

Reading Img

In [53]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x / 255.0
    return x

Readin Mask

In [54]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x / 250.0
    x = np.expand_dims(x, axis= -1)
    return x

tf.data pipeline

In [55]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x,y
    
    x,y = tf.numpy_function(_parse,[x,y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])

    return x, y

In [56]:
def tf_dataset(x, y , batch = 8):
    dataset = tf.data.Dataset.from_tensor_slices((x,y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

Training

In [57]:
(train_x, train_y),(valid_x, valid_y) = load_data(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 15 - 15
Valid: 5 - 5


In [58]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [59]:
for x, y in train_dataset:
    print(x.shape, y.shape)

: 

In [ ]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 180, 166, 512), (None, 181, 167, 512)]

In [ ]:
model.summary()

In [ ]:
opt = tf.keras.Optimizer.Adam(lr)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["acc"])

In [ ]:
callbacks = [
    ModelCheckpoint(model_file, verbose= 1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0,1,patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=False)
]

In [ ]:
model.fit(
    train_dataset,
    validation_data = valid_dataset,
    epochs = epochs,
    callbacks = callbacks
)